In [ ]:
%%capture
!pip install transformers
!pip install pypinyin
!pip install jieba
!pip install paddlepaddle

In [1]:
import re,time,json
from collections import defaultdict
from torch.utils.data import DataLoader
from pypinyin import pinyin, Style
from tqdm import tqdm
import pickle

import numpy as np
import torch
import torch.nn.functional as F
import torch.nn as nn
from copy import deepcopy

from transformers import (BertTokenizer,BertConfig,BertModel)

from model.Embedding import *
from model.fusionDataset import FusionDataset

import jieba
import jieba.posseg as pseg
import paddle

config = BertConfig.from_pretrained('AnchiBERT')
tokenizer = BertTokenizer.from_pretrained('AnchiBERT')
Anchibert = BertModel.from_pretrained('AnchiBERT',config=config)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

Some weights of the model checkpoint at AnchiBERT were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


### Load Necessary preproceeded Data

In [2]:
with open('data/char_map.json','r') as f:
    glyph2ix = defaultdict(lambda : 1)
    glyph2ix.update({'[CLS]':0,'[SEP]':0,'[PAD]':0})
    glyph2ix.update(json.load(f))

with open('data/pinyin_map.json','r') as f:
    pinyin2ix = defaultdict(lambda : 1)
    pinyin2ix.update({'[CLS]':0,'[SEP]':0,'[PAD]':0})
    pinyin2ix.update(json.load(f))
    
with open('data/pos_tags.json','r') as f:
    pos2ix = defaultdict(lambda : 0)
    pos2ix.update(json.load(f))

/opt/conda/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [3]:
# train 上联
with open("couplet/train/in.txt",encoding='utf8') as f:
    tr_in =  [row.strip().split() for row in f.readlines()]
# train 下联  
with open("couplet/train/out.txt",encoding='utf8') as f:
    tr_out = [row.strip().split() for row in f.readlines()]
with open('data/train_in_pos.pt','rb') as f:
    tr_pos_in = pickle.load(f)
with open('data/train_out_pos.pt','rb') as f:
    tr_pos_out = pickle.load(f)

In [4]:
display(len(tr_in))
total_len = len(tr_in)
half = total_len//1000
display(half)
train_split = int(0.95 * half)


770491

770

In [5]:
trainSet = FusionDataset(tr_in[:train_split],tokenizer,\
                         glyph2ix,pinyin2ix,pos2ix,tr_out[:train_split],\
                         tr_pos_in[:train_split],tr_pos_out[:train_split],\
                         device=device) # use device if you want to load it gpu

731it [00:00, 2292.82it/s]
731it [00:00, 2265.69it/s]


In [6]:
valSet = FusionDataset(tr_in[train_split:half],tokenizer,\
                       glyph2ix,pinyin2ix,pos2ix,tr_out[train_split:half],\
                       tr_pos_in[train_split:half],tr_pos_out[train_split:half],\
                       device=device) # use device if you want to load it gpu

39it [00:00, 2232.08it/s]
39it [00:00, 2271.76it/s]


In [7]:
from model.fusion_transformer import Fusion_Anchi_Trans_Decoder, Fusion_Anchi_Transformer, Anchi_Decoder,Anchi_Transformer

In [8]:
from utils.trans_trainer import train

In [9]:
config = { # for Fusion_Anchi_Trans_Decoder
    'max_position_embeddings':50,
    'hidden_size':768,
    'font_weight_path':'data/glyph_weight.npy',
    'pinyin_embed_dim':30, # trainable
    'pinyin_path':'data/pinyin_map.json',
    'tag_size':30,
    'tag_emb_dim':10, # trainable
    'layer_norm_eps':1e-12,
    'hidden_dropout':0.1,
    'nhead':12,
    'num_layers':6, # trainable
    'output_dim':21128 # fixed
}
config2 = { # for Anchi_Trans_Decoder
    'max_position_embeddings':50,
    'hidden_size':768,
    'layer_norm_eps':1e-12,
    'hidden_dropout':0.1,
    'nhead':12,
    'num_layers':6, # trainable
    'output_dim':21128 # fixed
}
config3 = { # Fusion_Anchi_Transformer
    'max_position_embeddings':50,
    'hidden_size':768,
    'font_weight_path':'data/glyph_weight.npy',
    'pinyin_embed_dim':30, # trainable
    'pinyin_path':'data/pinyin_map.json',
    'tag_size':30,
    'tag_emb_dim':10, # trainable 
    'layer_norm_eps':1e-12, 
    'hidden_dropout':0.1, 
    'nhead':12,
    'num_encoder_layers':6, # trainable
    'num_decoder_layers':6, # trainable
    'output_dim':21128, # fixed
    'dim_feedforward': 3072,
    'activation':'relu',
    'trans_dropout':0.1
}
config4 = { # Anchi_Transformer
    'max_position_embeddings':50,
    'hidden_size':768,
    'layer_norm_eps':1e-12, 
    'hidden_dropout':0.1, 
    'nhead':12,
    'num_encoder_layers':6, # trainable
    'num_decoder_layers':6, # trainable
    'output_dim':21128, # fixed
    'dim_feedforward': 3072,
    'activation':'relu',
    'trans_dropout':0.1
}

In [10]:
model = Fusion_Anchi_Trans_Decoder(config)

In [10]:
model = Anchi_Decoder(config2)

In [ ]:
model = Fusion_Anchi_Transformer(config3)

In [ ]:
model = Anchi_Transformer(config4)

In [10]:
# batch_size = [32,64,128]
# lr =[0.1,0.01,0.001]
# name = 'anchi_tra_de_128_01_10_'
name = 'test'
train(Fusion_Anchi_Trans_Decoder(config),trainSet,valSet,batch_size=128,lr=0.01,
      epoch=5,bert=Anchibert,name= name, with_trans=True,
      optimizer_name='Adam',scheduleFactor=0.5,
      schedule_Patience=5,min_lr=1e-06,verbose=True
      ,patience=10,store='result/')
torch.cuda.empty_cache()

  0%|          | 0/5 [00:00<?, ?it/s]

Epoch: 01 | Epoch Time: 0m 2s
	Training Loss: 11.43506 	Validation Loss: 13.64566


 20%|██        | 1/5 [00:03<00:14,  3.72s/it]

Epoch: 02 | Epoch Time: 0m 3s
	Training Loss: 9.86644 	Validation Loss: 9.27988


 40%|████      | 2/5 [00:07<00:11,  3.82s/it]

...

 60%|██████    | 3/5 [00:11<00:07,  3.64s/it]

Epoch: 04 | Epoch Time: 0m 2s
	Training Loss: 7.44927 	Validation Loss: 8.40546


 80%|████████  | 4/5 [00:14<00:03,  3.52s/it]

Epoch: 05 | Epoch Time: 0m 2s
	Training Loss: 7.07967 	Validation Loss: 7.90253


100%|██████████| 5/5 [00:17<00:00,  3.49s/it]


In [12]:
name = 'test'
train(Anchi_Decoder(config2),trainSet,valSet,batch_size=128,lr=0.01,
      epoch=5,bert=Anchibert,name= name, with_trans=True,
      optimizer_name='Adam',scheduleFactor=0.5,
      schedule_Patience=5,min_lr=1e-06,verbose=True
      ,patience=10,store='result/')
torch.cuda.empty_cache()

  0%|          | 0/5 [00:00<?, ?it/s]

Epoch: 01 | Epoch Time: 0m 2s
	Training Loss: 11.42733 	Validation Loss: 12.79454


 20%|██        | 1/5 [00:03<00:12,  3.12s/it]

Epoch: 02 | Epoch Time: 0m 2s
	Training Loss: 9.22428 	Validation Loss: 9.68234


 40%|████      | 2/5 [00:06<00:09,  3.12s/it]

...

 60%|██████    | 3/5 [00:09<00:06,  3.10s/it]

Epoch: 04 | Epoch Time: 0m 2s
	Training Loss: 7.46385 	Validation Loss: 8.50035


 80%|████████  | 4/5 [00:12<00:03,  3.12s/it]

Epoch: 05 | Epoch Time: 0m 2s
	Training Loss: 6.99139 	Validation Loss: 8.25432


100%|██████████| 5/5 [00:15<00:00,  3.10s/it]


In [13]:
name = 'test'
train(Fusion_Anchi_Transformer(config3),trainSet,valSet,batch_size=128,lr=0.01,
      epoch=5,bert=Anchibert,name= name, with_trans=True,
      optimizer_name='Adam',scheduleFactor=0.5,
      schedule_Patience=5,min_lr=1e-06,verbose=True
      ,patience=10,store='result/')
torch.cuda.empty_cache()

  0%|          | 0/5 [00:00<?, ?it/s]

Epoch: 01 | Epoch Time: 0m 3s
	Training Loss: 10.81354 	Validation Loss: 11.50086


 20%|██        | 1/5 [00:04<00:18,  4.54s/it]

Epoch: 02 | Epoch Time: 0m 3s
	Training Loss: 8.77467 	Validation Loss: 8.95470


 40%|████      | 2/5 [00:09<00:13,  4.53s/it]

...

 60%|██████    | 3/5 [00:13<00:09,  4.52s/it]

Epoch: 04 | Epoch Time: 0m 3s
	Training Loss: 7.30146 	Validation Loss: 8.27623


 80%|████████  | 4/5 [00:18<00:04,  4.53s/it]

Epoch: 05 | Epoch Time: 0m 3s
	Training Loss: 6.95746 	Validation Loss: 7.89394


100%|██████████| 5/5 [00:22<00:00,  4.54s/it]


In [14]:
name = 'test'
train(Anchi_Transformer(config4),trainSet,valSet,batch_size=128,lr=0.01,
      epoch=5,bert=Anchibert,name= name, with_trans=True,
      optimizer_name='Adam',scheduleFactor=0.5,
      schedule_Patience=5,min_lr=1e-06,verbose=True
      ,patience=10,store='result/')
torch.cuda.empty_cache()

  0%|          | 0/5 [00:00<?, ?it/s]

Epoch: 01 | Epoch Time: 0m 3s
	Training Loss: 10.88587 	Validation Loss: 11.88156


 20%|██        | 1/5 [00:04<00:17,  4.37s/it]

Epoch: 02 | Epoch Time: 0m 3s
	Training Loss: 8.89104 	Validation Loss: 9.07632


 40%|████      | 2/5 [00:08<00:13,  4.40s/it]

...

 60%|██████    | 3/5 [00:13<00:08,  4.41s/it]

Epoch: 04 | Epoch Time: 0m 3s
	Training Loss: 7.37130 	Validation Loss: 8.13629


 80%|████████  | 4/5 [00:17<00:04,  4.41s/it]

Epoch: 05 | Epoch Time: 0m 3s
	Training Loss: 6.96033 	Validation Loss: 7.84530


100%|██████████| 5/5 [00:22<00:00,  4.43s/it]
